In [1]:
import pandas as pd
import geohash_hilbert as ghh
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import os, json, requests, pickle
from scipy.stats import skew
from shapely.geometry import Point,Polygon,MultiPoint,MultiPolygon
from scipy.stats import ttest_ind, f_oneway, lognorm, levy, skew, chisquare
#import scipy.stats as st
from sklearn.preprocessing import normalize, scale
from tabulate import tabulate #pretty print of tables. source: http://txt.arboreus.com/2013/03/13/pretty-print-tables-in-python.html
from shapely.geometry import Point,Polygon,MultiPoint
%matplotlib inline

In [2]:
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [4]:
from sklearn.metrics import mean_squared_error

In [5]:
data = pd.read_csv('final_clean_data_201501_06.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.head(3)

,Unnamed: 0,dayofweek,dayofyear,day_and_hour,pickups,geohash_pickup,FHV_count,DATE,DAILYSunrise,DAILYSunset,...,temperature_max,temperature_min,fastest_2minute_windspeed,fog,heavy_fog,ice_pellets,glaze_rime,smoke_haze,EVENT_TYPE,month
0,0,Thursday,29,2914,1,SHBHR,4252,2015-01-29,709,1710,...,36.0,19.0,14.1,0.0,0.0,0.0,0.0,0.0,-1,1
1,1,Wednesday,21,2111,1,SHBI3,3456,2015-01-21,715,1700,...,36.0,25.0,10.1,0.0,0.0,0.0,0.0,0.0,-1,1
2,2,Saturday,10,109,1,SHBJ3,3027,2015-01-10,720,1647,...,23.0,16.0,18.1,0.0,0.0,0.0,0.0,0.0,-1,1


In [7]:
data['dayofweek'] = pd.factorize(data['dayofweek'])[0]

In [8]:
Xnames = ["dayofweek", "dayofyear", "day_and_hour", "geohash_pickup", "FHV_count", "DAILYSunrise",
          "DAILYSunset", "wind_speed_average", "precipitation", "snowfall", "snow_depth",
         'temperature_max', 'temperature_min', 'fastest_2minute_windspeed', 'fog', 'ice_pellets',
         'smoke_haze', 'month']
X = data[Xnames]

In [9]:
# Get the longitude and latitude from the geohash
def decodegeo(geo, which):
    if len(geo) >= 5:
        geodecoded = ghh.decode(geo)
        return geodecoded[which]
    else:
        return 0
X['latitude'] = X['geohash_pickup'].apply(lambda geo: decodegeo(geo, 0))
X['longitude'] = X['geohash_pickup'].apply(lambda geo: decodegeo(geo, 1))
X.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,dayofweek,dayofyear,day_and_hour,geohash_pickup,FHV_count,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,ice_pellets,smoke_haze,month,latitude,longitude
0,0,29,2914,SHBHR,4252,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,-74.053345,40.855408
1,1,21,2111,SHBI3,3456,715,1700,4.25,0.00,0.0,0.0,36.0,25.0,10.1,0.0,0.0,0.0,1,-74.009399,40.816956
2,2,10,109,SHBJ3,3027,720,1647,8.05,0.00,0.0,1.0,23.0,16.0,18.1,0.0,0.0,0.0,1,-73.987427,40.783997
3,3,11,116,SHBJ3,4476,720,1649,6.04,0.00,0.0,1.0,37.0,18.0,14.1,0.0,0.0,0.0,1,-73.987427,40.783997
4,4,12,1210,SHBJ3,4176,719,1650,3.80,0.36,0.0,0.0,39.0,35.0,12.1,1.0,1.0,0.0,1,-73.987427,40.783997


In [10]:
X = X.drop(['geohash_pickup'], axis=1)

In [11]:
X = X.drop(['day_and_hour'], axis=1)

In [12]:
y = data.pickups

In [13]:
itrain, itest = train_test_split(range(data.shape[0]), train_size=0.8)
mask=np.ones(data.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

array([ True, False, False,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [14]:
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
n_samples = Xtrain.shape[0]
n_features = Xtrain.shape[1]
print (Xtrain.shape)
max_samples = 1000000
if Xtrain.shape[0] > max_samples:
    rows = random.sample(Xtrain.index, max_samples)
    Xtrain = Xtrain.ix[rows]
    ytrain = ytrain.ix[rows]
print (Xtrain.shape)
Xtrain.head()

(126616, 18)
(126616, 18)


,dayofweek,dayofyear,FHV_count,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,ice_pellets,smoke_haze,month,latitude,longitude
0,0,29,4252,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,-74.053345,40.855408
3,3,11,4476,720,1649,6.04,0.00,0.0,1.0,37.0,18.0,14.1,0.0,0.0,0.0,1,-73.987427,40.783997
4,4,12,4176,719,1650,3.80,0.36,0.0,0.0,39.0,35.0,12.1,1.0,1.0,0.0,1,-73.987427,40.783997
5,4,12,5484,719,1650,3.80,0.36,0.0,0.0,39.0,35.0,12.1,1.0,1.0,0.0,1,-73.987427,40.783997
6,4,12,1159,719,1650,3.80,0.36,0.0,0.0,39.0,35.0,12.1,1.0,1.0,0.0,1,-73.987427,40.783997


In [15]:
itrain, itest = train_test_split(range(data.shape[0]), train_size=0.8)
mask=np.ones(data.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [16]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None, verbose=0):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func, verbose=verbose)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds, verbose=verbose)
    gs.fit(X, y)
    print ("BEST", gs.best_params_, gs.best_score_, gs.grid_scores_, gs.scorer_)
    print ("Best score: ", gs.best_score_)
    best = gs.best_estimator_
    return best

In [17]:
estimator = RandomForestRegressor(n_estimators=30, n_jobs=-1)

In [18]:
%%time
# Define a grid of parameters over which to optimize the random forest
# We will figure out which number of trees is optimal
parameters = {"n_estimators": [50],
              "max_features": ["auto"], # ["auto","sqrt","log2"]
              "max_depth": [50]}
best = cv_optimize(estimator, parameters, Xtrain, ytrain, n_folds=5, score_func='mean_squared_error', verbose=3)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.0s remaining:    0.0s


[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-3.430723 -   9.0s
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   18.1s remaining:    0.0s


[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-5.024174 -   9.0s
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


...

[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-3.642370 -   9.2s
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-3.553929 -   9.9s
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   47.9s finished


[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-3.222859 -  10.6s
BEST {'max_depth': 50, 'max_features': 'auto', 'n_estimators': 50} -3.7748084752726374 [mean: -3.77481, std: 0.64036, params: {'max_depth': 50, 'max_features': 'auto', 'n_estimators': 50}] make_scorer(mean_squared_error, greater_is_better=False)
Best score:  -3.7748084752726374
CPU times: user 3min 24s, sys: 2.72 s, total: 3min 27s
Wall time: 1min


In [19]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
reg=best.fit(Xtrain, ytrain)
training_accuracy = reg.score(Xtrain, ytrain)
test_accuracy = reg.score(Xtest, ytest)
print ("############# based on standard predict ################")
print ("R^2 on training data: %0.4f" % (training_accuracy))
print ("R^2 on test data:     %0.4f" % (test_accuracy))

############# based on standard predict ################
R^2 on training data: 0.9372
R^2 on test data:     0.5727


In [20]:
# Calculate the Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(reg.predict(Xtest),ytest))
print ("RMSE = %0.3f (this is in log-space!)" % rmse)
print ("So two thirds of the records would be a factor of less than %0.2f away from the real value." % np.power(10,rmse))

RMSE = 1.799 (this is in log-space!)
So two thirds of the records would be a factor of less than 63.02 away from the real value.


In [21]:
X.head()

,dayofweek,dayofyear,FHV_count,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,ice_pellets,smoke_haze,month,latitude,longitude
0,0,29,4252,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,-74.053345,40.855408
1,1,21,3456,715,1700,4.25,0.00,0.0,0.0,36.0,25.0,10.1,0.0,0.0,0.0,1,-74.009399,40.816956
2,2,10,3027,720,1647,8.05,0.00,0.0,1.0,23.0,16.0,18.1,0.0,0.0,0.0,1,-73.987427,40.783997
3,3,11,4476,720,1649,6.04,0.00,0.0,1.0,37.0,18.0,14.1,0.0,0.0,0.0,1,-73.987427,40.783997
4,4,12,4176,719,1650,3.80,0.36,0.0,0.0,39.0,35.0,12.1,1.0,1.0,0.0,1,-73.987427,40.783997


In [22]:
time_cols = list(X.columns.values)
time_cols

['dayofweek',
 'dayofyear',
 'FHV_count',
 'DAILYSunrise',
 'DAILYSunset',
 'wind_speed_average',
 'precipitation',
 'snowfall',
 'snow_depth',
 'temperature_max',
 'temperature_min',
 'fastest_2minute_windspeed',
 'fog',
 'ice_pellets',
 'smoke_haze',
 'month',
 'latitude',
 'longitude']

In [23]:
# Construct dataframes with all possible times (time_data) and all possible locations (loc_data)

# Columns about time
time_cols = list(X.columns.values)
time_cols.remove('latitude')
time_cols.remove('longitude')
#time_cols.remove('snowfall')
#time_cols.remove('snow_depth')
#time_cols.remove('temperature_max')
#time_cols.remove('temperature_min')
#time_cols.remove('fastest_2minute_windspeed')
#time_cols.remove('fog')
#time_cols.remove('ice_pellets')
#time_cols.remove('smoke_haze')
#time_cols.remove('precipitation')
#time_cols.remove('DAILYSunset')
#time_cols.remove('DAILYSunrise')

# Columns about location
loc_cols = ['latitude', 'longitude']








In [24]:
# Unique times
time_data = X.drop(loc_cols, axis=1).drop_duplicates()

In [25]:
time_data

,dayofweek,dayofyear,FHV_count,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,ice_pellets,smoke_haze,month
0,0,29,4252,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1
1,1,21,3456,715,1700,4.25,0.00,0.0,0.0,36.0,25.0,10.1,0.0,0.0,0.0,1
2,2,10,3027,720,1647,8.05,0.00,0.0,1.0,23.0,16.0,18.1,0.0,0.0,0.0,1
3,3,11,4476,720,1649,6.04,0.00,0.0,1.0,37.0,18.0,14.1,0.0,0.0,0.0,1
4,4,12,4176,719,1650,3.80,0.36,0.0,0.0,39.0,35.0,12.1,1.0,1.0,0.0,1
5,4,12,5484,719,1650,3.80,0.36,0.0,0.0,39.0,35.0,12.1,1.0,1.0,0.0,1
6,4,12,1159,719,1650,3.80,0.36,0.0,0.0,39.0,35.0,12.1,1.0,1.0,0.0,1
7,5,13,2590,719,1651,8.50,0.00,0.0,0.0,36.0,17.0,18.1,0.0,0.0,0.0,1
8,1,14,3782,719,1652,6.26,0.00,0.0,0.0,32.0,16.0,14.1,0.0,0.0,0.0,1
9,0,15,1741,718,1653,4.70,0.00,0.0,0.0,35.0,25.0,15.0,0.0,0.0,0.0,1


In [26]:
# In Tableau we are only going to look at Monday
time_data = time_data[time_data['dayofweek'] == 0]
time_data

,dayofweek,dayofyear,FHV_count,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,ice_pellets,smoke_haze,month
0,0,29,4252,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1
9,0,15,1741,718,1653,4.70,0.00,0.0,0.0,35.0,25.0,15.0,0.0,0.0,0.0,1
22,0,22,4012,715,1701,5.59,0.00,0.0,0.0,40.0,31.0,14.1,0.0,0.0,1.0,1
34,0,29,4066,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1
35,0,29,1112,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1
58,0,1,7869,720,1639,7.16,0.00,0.0,0.0,39.0,27.0,17.0,0.0,0.0,0.0,1
59,0,1,6689,720,1639,7.16,0.00,0.0,0.0,39.0,27.0,17.0,0.0,0.0,0.0,1
69,0,1,5521,720,1639,7.16,0.00,0.0,0.0,39.0,27.0,17.0,0.0,0.0,0.0,1
70,0,1,4654,720,1639,7.16,0.00,0.0,0.0,39.0,27.0,17.0,0.0,0.0,0.0,1
71,0,1,4208,720,1639,7.16,0.00,0.0,0.0,39.0,27.0,17.0,0.0,0.0,0.0,1


In [27]:
# Unique locations
loc_data = X.drop(time_cols, axis=1).drop_duplicates()
loc_data

,latitude,longitude
0,-74.053345,40.855408
1,-74.009399,40.816956
2,-73.987427,40.783997
50,-73.976440,40.783997
383,-73.965454,40.783997
467,-73.965454,40.789490
621,-73.976440,40.789490
885,-73.976440,40.794983
1062,-73.965454,40.794983
1232,-73.965454,40.800476


In [28]:
# Dummy column to be able to join them together
time_data['key'] = 1
loc_data['key'] = 1

# Merge the time_data and location_data
result = pd.merge(time_data, loc_data, on='key').drop(['key'], axis=1)
result = result[Xtrain.columns.values]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
time_data.head()

,dayofweek,dayofyear,FHV_count,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,ice_pellets,smoke_haze,month,key
0,0,29,4252,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,1
9,0,15,1741,718,1653,4.70,0.00,0.0,0.0,35.0,25.0,15.0,0.0,0.0,0.0,1,1
22,0,22,4012,715,1701,5.59,0.00,0.0,0.0,40.0,31.0,14.1,0.0,0.0,1.0,1,1
34,0,29,4066,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,1
35,0,29,1112,709,1710,3.80,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,1


In [30]:
loc_data.head()

,latitude,longitude,key
0,-74.053345,40.855408,1
1,-74.009399,40.816956,1
2,-73.987427,40.783997,1
50,-73.976440,40.783997,1
383,-73.965454,40.783997,1


In [31]:
result.head(3)

,dayofweek,dayofyear,FHV_count,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,ice_pellets,smoke_haze,month,latitude,longitude
0,0,29,4252,709,1710,3.8,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,-74.053345,40.855408
1,0,29,4252,709,1710,3.8,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,-74.009399,40.816956
2,0,29,4252,709,1710,3.8,0.02,0.0,6.0,36.0,19.0,14.1,0.0,0.0,0.0,1,-73.987427,40.783997


In [32]:
# Get the real number of pickups and take care that we can merge it with the predictions,
# by also taking the geohash and the timestamp
yy = data[['geohash_pickup','dayofyear','pickups']]

# Decode the geohash in the latitude and longitude
yy['latitude'] = yy['geohash_pickup'].apply(lambda geo: decodegeo(geo, 0))
yy['longitude'] = yy['geohash_pickup'].apply(lambda geo: decodegeo(geo, 1))

# Do predictions and convert the logarithm to the normal numbers
result['pred_pickups'] = np.power(10,reg.predict(result)) - 1

# Merge the predictions and the real pickups
result = pd.merge(result, yy, how='left', on=['day_num', 'latitude', 'longitude']).drop(['geohash'], axis=1)
result.head(10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


KeyError: 'day_num'